# Example string workflow
Entire workflow from parsing ODE to generating approximation

In [1]:
from ODE_parser import ODE_parser
from approximation import ODE_approx
import sympy
import numpy as np
import matplotlib.pyplot as plt

User inputs:
- init_vals: initial values
- M, L: maximum of second and first derivatives of input function
- ODE: string form of an ODE, may be coupled ODEs 
- ODE_actual (optional): string form of the actual solution

This example is of the differential equation
$$\{x'=v, v'=9.8 \& t \le 1\}$$
where $\{ t ' = 1 \}$ is implicitly assumed.

In [2]:
x0 = 0.3
v0 = 0.9
init_vals = [x0, v0]

M = 10
L = 10

ODE = " { x' = v, v' = 9.8 & t <= 1 }"
ODE_actual = "[x0 + v0*t + 1/2 * 9.8 * t**2, v0 + 9.8*t]"

# x0 = 1
# v0 = 0
# init_vals = [x0, v0]
# ODE = "{x'=-y, y'=x & t <= 2*3.1416}"
# ODE_actual = "[cos(t), sin(t)]"

Parsing ODE_actual and ODE and identifying the ODE variables. 

ODE_actual_fn([x0, v0], t) is a lambda function representing the actual solution as a function of initial variables and time.

ODE_parser generates
- res(bool): if the parsing was successful or not
- vars(array of sympy.symbols): ODE variables identified
- fn(python function): function of the form fn(t, array_of_vars) where x may be a vector and is the RHS of the differential equation
$$\{x' = fn(t, x)\}$$
- T(float): maximum time the ODE runs

In [3]:
ODE_actual_fn = sympy.lambdify([sympy.symbols("x0 v0"), sympy.symbols("t")], sympy.sympify(ODE_actual))

res, vars, fn, T = ODE_parser(ODE)
if not res:
    print("Parsing failed")
    raise Exception("Parsing Failed")
print("ode variables: ", vars)
print("maximum time: ", T)

ode variables:  [x, y]
maximum time:  6.28320000000000


Example syntax of the parsed fn in terms of inputs and outputs. Here, $t = 0.1$, $x = (1,0)$ and the function returns a vector.

In [4]:
fn(0.1, (1, 0))

array([[0],
       [1]])

Generating the ODE approximating function.

In [5]:
ODE_approx_fn = ODE_approx(fn, len(vars), T, h=0.1)

In [6]:
print(ODE_actual_fn(init_vals, 0.1))
print(ODE_approx_fn(init_vals, 0.1))

[0.9950041652780258, 0.09983341664682815]
[array(1.), array(0.1)]


Plotting a comparison between the actual and approximated functions for each of the ODE variables/dimensions of the vector differential equation.

In [7]:
tt = np.linspace(0, T)
for i in range(len(vars)):
    x_actual = [ODE_actual_fn(init_vals, t)[i] for t in tt]
    x_approx = [ODE_approx_fn(init_vals, t)[i] for t in tt]
    plt.plot(tt, x_actual, label="actual "+str(vars[i]))  
    plt.plot(tt, x_approx, label="approx "+str(vars[i]))
    plt.legend()
    plt.title("Actual vs Approx for "+str(vars[i]))
    print(max(abs(np.array(x_actual)-np.array(x_approx))))

TypeError: Cannot interpret '6.28320000000000' as a data type